# Deep Learning

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
from matplotlib .pyplot import subplots

# Scikit-Learn
from sklearn. pipeline import Pipeline
from sklearn. model_selection import KFold
from sklearn. preprocessing import StandardScaler
from sklearn. model_selection import (train_test_split, GridSearchCV)
from sklearn. linear_model import (LinearRegression, LogisticRegression, Lasso)

# ISLP
from ISLP import load_data
from ISLP.models import ModelSpec as MS

In [ ]:
#### Torch-Specific Imports

In [ ]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset